# Setup

In [127]:
import logging
import math

import torch
import torch.nn.functional as F
from torch import nn

#
# From StableDynamics.ipynb; Zico Kolter
#

# You can use this to compensate for numeric error:
VERIFY = False
V_SCALE = 0.01

global V_WRAP, SCALE_FX
V_WRAP = False
SCALE_FX = False

class Dynamics(nn.Module):
    def __init__(self, fhat, V, alpha=0.01):
        super().__init__()
        self.fhat = fhat
        self.V = V
        self.alpha = 0 # alpha

    def forward(self, x):
        fx = self.fhat(x)
        if SCALE_FX:
            fx = fx / fx.norm(p=2, dim=1, keepdim=True).clamp(min=1.0)

        Vx = self.V(x)
        gV = torch.autograd.grad([a for a in Vx], [x], create_graph=True, only_inputs=True)[0]
        rv = fx - gV * (F.relu((gV*fx).sum(dim=1) + self.alpha*Vx[:,0])/(gV**2).sum(dim=1))[:,None]

        if VERIFY:
            # Verify that rv has no positive component along gV.
            # This helps us catch:
            #   (1) numeric error in the symbolic gradient calculation, and
            #   (2) Violation of the Lyapunov function when Euler integration is used.
            verify = (gV * rv).sum(dim=1)
            num_violation = len([v for v in verify if v > 0]) # (1)
            new_V = self.V(x + V_SCALE * rv)
            if (new_V > Vx).any(): # (2)
                err = sorted([v for v in (new_V - Vx).detach().cpu().numpy().ravel() if v > 0], reverse=True)

        return rv

class ICNN(nn.Module):
    def __init__(self, layer_sizes, activation=F.relu_):
        super().__init__()
        self.W = nn.ParameterList([nn.Parameter(torch.Tensor(l, layer_sizes[0])) 
                                   for l in layer_sizes[1:]])
        self.U = nn.ParameterList([nn.Parameter(torch.Tensor(layer_sizes[i+1], layer_sizes[i]))
                                   for i in range(1,len(layer_sizes)-1)])
        self.bias = nn.ParameterList([nn.Parameter(torch.Tensor(l)) for l in layer_sizes[1:]])
        self.act = activation
        self.reset_parameters()

    def reset_parameters(self):
        # copying from PyTorch Linear
        for W in self.W:
            nn.init.kaiming_uniform_(W, a=5**0.5)
        for U in self.U:
            nn.init.kaiming_uniform_(U, a=5**0.5)
        for i,b in enumerate(self.bias):
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.W[i])
            bound = 1 / (fan_in**0.5)
            nn.init.uniform_(b, -bound, bound)

    def forward(self, x):
        z = F.linear(x, self.W[0], self.bias[0])
        z = self.act(z)

        for W,b,U in zip(self.W[1:-1], self.bias[1:-1], self.U[:-1]):
            z = F.linear(x, W, b) + F.linear(z, F.softplus(U)) / U.shape[0]
            z = self.act(z)

        return F.linear(x, self.W[-1], self.bias[-1]) + F.linear(z, F.softplus(self.U[-1])) / self.U[-1].shape[0]



class ReHU(nn.Module):
    """ Rectified Huber unit"""
    def __init__(self, d):
        super().__init__()
        self.a = 1/d
        self.b = -d/2

    def forward(self, x):
        return torch.max(torch.clamp(torch.sign(x)*self.a/2*x**2,min=0,max=-self.b),x+self.b)

class MakePSD(nn.Module):
    def __init__(self, f, n, eps=0.01, d=1.0):
        super().__init__()
        self.f = f
        self.zero = torch.nn.Parameter(f(torch.zeros(1,n)), requires_grad=False)
        self.eps = eps
        self.d = d
        self.rehu = ReHU(self.d)

    def forward(self, x):
        smoothed_output = self.rehu(self.f(x) - self.zero)
        quadratic_under = self.eps*(x**2).sum(1,keepdim=True)
        return smoothed_output + quadratic_under

class PosDefICNN(nn.Module):
    def __init__(self, layer_sizes, eps=0.1, negative_slope=0.05):
        super().__init__()
        self.W = nn.ParameterList([nn.Parameter(torch.Tensor(l, layer_sizes[0])) 
                                   for l in layer_sizes[1:]])
        self.U = nn.ParameterList([nn.Parameter(torch.Tensor(layer_sizes[i+1], layer_sizes[i]))
                                   for i in range(1,len(layer_sizes)-1)])
        self.eps = eps
        self.negative_slope = negative_slope
        self.reset_parameters()

    def reset_parameters(self):
        # copying from PyTorch Linear
        for W in self.W:
            nn.init.kaiming_uniform_(W, a=5**0.5)
        for U in self.U:
            nn.init.kaiming_uniform_(U, a=5**0.5)

    def forward(self, x):
        z = F.linear(x, self.W[0])
        F.leaky_relu_(z, negative_slope=self.negative_slope)

        for W,U in zip(self.W[1:-1], self.U[:-1]):
            z = F.linear(x, W) + F.linear(z, F.softplus(U))*self.negative_slope
            z = F.leaky_relu_(z, negative_slope=self.negative_slope)

        z = F.linear(x, self.W[-1]) + F.linear(z, F.softplus(self.U[-1]))
        return F.relu(z) + self.eps*(x**2).sum(1)[:,None]

def loss(Ypred, Yactual, X):
    # Force smoothness in V:
    # penalty for new_V being larget than old V:
    Vloss = torch.tensor(0)
    if SMOOTH_V:
        V = model.V
        # Successor to X:
        succ_X = (X + V_SCALE * Yactual).detach()
        if V_WRAP:
            while torch.any(succ_X < -math.pi):
                succ_X[succ_X < -math.pi] = succ_X[succ_X < -math.pi] + 2 * math.pi
            while torch.any(succ_X >= math.pi):
                succ_X[succ_X >= math.pi] = succ_X[succ_X >= math.pi] - 2 * math.pi
            succ_X.requires_grad_()

        Vloss = (V(succ_X) - V(X)).clamp(min=0).mean()

    l2loss = ((Ypred - Yactual)**2).mean()

    return (l2loss + SMOOTH_V * Vloss, l2loss, Vloss)

global model, SMOOTH_V
model = None
SMOOTH_V = 0

def loss_flatten(l):
    return l

def loss_labels():
    return ["loss", "l2", "V"]

def summary(*a, **kw):
    pass

# Verification

In [128]:
lsd = 2
ph_dim = 32
h_dim = 32

fhat = nn.Sequential(nn.Linear(lsd, h_dim), nn.ReLU(),
                        nn.Linear(h_dim, h_dim), nn.ReLU(),
                        nn.Linear(h_dim, lsd))

rehu = 0.005
projfn_eps = 0.01

V = MakePSD(ICNN([lsd, ph_dim, ph_dim, 1], activation=ReHU(float(0.005))), lsd, eps=projfn_eps, d=1.0)
model = Dynamics(fhat, V, alpha=0)


In [129]:
print(V.f.W[0])

Parameter containing:
tensor([[-0.4469, -0.4674],
        [ 0.6757,  0.3579],
        [ 0.6619, -0.4120],
        [-0.1822, -0.1744],
        [ 0.6781,  0.1104],
        [ 0.6598,  0.0854],
        [ 0.3903, -0.6208],
        [ 0.2241, -0.2727],
        [ 0.3835, -0.2834],
        [ 0.1877,  0.4991],
        [ 0.0467, -0.6619],
        [-0.3418,  0.3914],
        [ 0.4909,  0.2923],
        [ 0.2872,  0.2930],
        [ 0.6654, -0.0721],
        [-0.0943,  0.1018],
        [ 0.6542, -0.1733],
        [-0.2382,  0.2452],
        [ 0.2350, -0.5035],
        [ 0.6242,  0.5085],
        [-0.6930,  0.2493],
        [ 0.0752, -0.6672],
        [ 0.6103,  0.2826],
        [-0.6369, -0.5072],
        [-0.4908, -0.2130],
        [ 0.3746, -0.6606],
        [ 0.5892, -0.3206],
        [ 0.5117,  0.4705],
        [ 0.0100,  0.3005],
        [ 0.6436,  0.5790],
        [ 0.2267, -0.5287],
        [-0.6910, -0.5193]], requires_grad=True)


In [130]:
model.load_state_dict(torch.load("trained_models/checkpoint_10000.pth"))
model.eval()

Dynamics(
  (fhat): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
  (V): MakePSD(
    (f): ICNN(
      (W): ParameterList(
          (0): Parameter containing: [torch.float32 of size 32x2]
          (1): Parameter containing: [torch.float32 of size 32x2]
          (2): Parameter containing: [torch.float32 of size 1x2]
      )
      (U): ParameterList(
          (0): Parameter containing: [torch.float32 of size 32x32]
          (1): Parameter containing: [torch.float32 of size 1x32]
      )
      (bias): ParameterList(
          (0): Parameter containing: [torch.float32 of size 32]
          (1): Parameter containing: [torch.float32 of size 32]
          (2): Parameter containing: [torch.float32 of size 1]
      )
      (act): ReHU()
    )
    (rehu): ReHU()
  )
)

# Load trajectory

In [131]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from scipy.integrate import odeint


FILE = "datasets/p-physics-1.npy"
N = 1

data = np.load(FILE)
print(f"Loaded data: {data.shape}")
selected_data = data[:,1,:]

# Uncomment to use the training dataset
load = np.load("datasets/p-1-test.npz")
selected_data = load["X"] 

Loaded data: (1000, 100, 2)


In [132]:
evals = []

x_plot = []
y_plot = []
for i in range(len(selected_data)):
    input = torch.FloatTensor([selected_data[i]])
    input.requires_grad = True
    x_plot.append(input[0][0].item())
    y_plot.append(input[0][1].item())
    energy_val = model.V(input).detach().item()
    evals.append(energy_val)

evals

[11.746335983276367,
 15.005767822265625,
 2.7048211097717285,
 10.214747428894043,
 3.0838751792907715,
 4.972617149353027,
 15.660563468933105,
 16.227603912353516,
 13.308359146118164,
 2.5354485511779785,
 6.2597222328186035,
 4.1421356201171875,
 20.576526641845703,
 14.605072021484375,
 12.229253768920898,
 14.718011856079102,
 20.469141006469727,
 3.354281187057495,
 1.9854999780654907,
 9.42226791381836,
 9.59760856628418,
 15.439620971679688,
 5.176284313201904,
 7.946085453033447,
 10.16596794128418,
 14.012321472167969,
 14.56551742553711,
 18.5009708404541,
 1.2133963108062744,
 0.15944960713386536,
 20.346086502075195,
 18.894550323486328,
 9.920496940612793,
 19.785247802734375,
 1.0365995168685913,
 15.588371276855469,
 4.7946858406066895,
 19.499420166015625,
 4.239372253417969,
 10.511611938476562,
 5.574895858764648,
 8.617895126342773,
 7.343969821929932,
 20.106643676757812,
 18.221967697143555,
 14.131528854370117,
 9.569811820983887,
 10.761448860168457,
 1.834026

In [133]:
from matplotlib import pyplot as plt
fig = plt.figure()
 
# syntax for 3-D plotting
ax = plt.axes(projection='3d')
 
# syntax for plotting
ax.plot_trisurf(x_plot, y_plot, evals, cmap='viridis',\
                edgecolor='green')
ax.set_title('Surface plot')
plt.show()

In [134]:
import plotly.graph_objects as go
import numpy as np

# Assuming your x_plot, y_plot, and evals are lists of 199 elements each
x_plot = np.array(x_plot)
y_plot = np.array(y_plot)
evals = np.array(evals)


fig = go.Figure(
    data=[go.Scatter3d(x=x_plot, y=y_plot, z=evals, mode='markers')]
)
fig.update_layout(
    title='V(x) outputs',
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=65, r=50, b=65, t=90),
    scene_aspectmode='cube'
)
fig.show()


In [135]:
len(evals)

200

In [136]:
load = np.load("datasets/p-1-train.npz")
X = load["X"]
np.shape(X)

(200, 2)